In [1]:
import os
import config
v1, v2, v3 = config.spark_env("msi")
os.environ['SPARK_VERSION'] = v1
os.environ['JAVA_HOME'] = v2
os.environ['SPARK_HOME'] = v3
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable
import numpy as np
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)
import spark_functions

In [2]:
india_spark = spark_functions.merged_df("India")

In [3]:
china_spark = spark_functions.merged_df("China")

In [4]:
germany_spark = spark_functions.merged_df("Germany")

In [5]:
france_spark = spark_functions.merged_df("France")

In [6]:
mexico_spark = spark_functions.merged_df("Mexico")

In [7]:
us_gdp = spark_functions.sql_to_spark_gdp("United States")

In [8]:
us_emissions = spark_functions.sql_to_spark_emissions("United States")

In [9]:
us_export, us_import = spark_functions.sql_to_spark_trade("USA")

In [10]:
us_spark = spark_functions.merge_sparks(us_import,us_export,us_gdp,us_emissions)

In [12]:
us_spark.columns

['country',
 'year',
 'gdp',
 'annual_co2_emissions_tonnes',
 'import_trade_sum_usd',
 'import_weight_sum_kg',
 'import_quantity_sum',
 'export_trade_sum_usd',
 'export_weight_sum_kg',
 'export_quantity_sum']

In [18]:
arr = np.array(us_spark.select(
 'gdp',
 'import_trade_sum_usd',
 'import_weight_sum_kg',
 'import_quantity_sum',
 'export_trade_sum_usd',
 'export_weight_sum_kg',
 'export_quantity_sum') \
.collect())

In [17]:
arr2 = np.array(us_spark.select('annual_co2_emissions_tonnes').collect())

In [19]:
arr

array([[2.43422589e+04, 5.53057798e+11, 4.28189156e+10, 5.05684280e+10,
        5.07650351e+11, 1.02364560e+11, 3.90458765e+11],
       [2.54189908e+04, 6.01355275e+11, 4.30612905e+10, 5.44852566e+10,
        5.38062351e+11, 1.28601941e+11, 3.60206509e+11],
       [2.63872937e+04, 6.52373937e+11, 4.81886924e+10, 7.31106729e+10,
        5.48398969e+11, 1.24705618e+11, 3.27010898e+11],
       [2.76948534e+04, 7.42750614e+11, 5.22685068e+10, 5.81304052e+10,
        5.98400239e+11, 1.15502837e+11, 3.55381908e+11],
       [2.86908757e+04, 8.30259504e+11, 5.24553446e+10, 5.85399640e+10,
        6.73915108e+11, 1.48562056e+11, 3.90540632e+11],
       [2.99677127e+04, 8.82586203e+11, 5.46499525e+10, 6.71688652e+10,
        7.23150044e+11, 1.33943029e+11, 4.01946693e+11],
       [3.14591390e+04, 9.70520718e+11, 5.83567132e+10, 6.83003622e+10,
        7.94208859e+11, 1.20356314e+11, 3.59007136e+11],
       [3.28536770e+04, 1.02755704e+12, 5.65830103e+10, 6.94122253e+10,
        7.97497195e+11, 1

In [20]:
arr2

array([[5.07083940e+09],
       [5.17405908e+09],
       [5.28468788e+09],
       [5.37798714e+09],
       [5.43921331e+09],
       [5.62618672e+09],
       [5.70389071e+09],
       [5.75232419e+09],
       [5.83225018e+09],
       [6.00060607e+09],
       [5.90245499e+09],
       [5.94365128e+09],
       [5.99131957e+09],
       [6.10662074e+09],
       [6.13200570e+09],
       [6.05223629e+09],
       [6.13098356e+09],
       [5.93277528e+09],
       [5.49539496e+09],
       [5.70107581e+09],
       [5.57070656e+09],
       [5.36673028e+09],
       [5.51961256e+09],
       [5.56875926e+09],
       [5.42080413e+09],
       [5.31086141e+09]])

In [21]:
arr.shape

(26, 7)

In [22]:
arr2.shape

(26, 1)

In [23]:
from sklearn.model_selection import train_test_split


In [24]:
X_train, X_test, y_train, y_test = train_test_split(arr, arr2)

In [25]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [26]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# def build_and_compile_model(norm):
#     model = keras.Sequential([
#       norm,
#       layers.Dense(64, activation='relu'),
#       layers.Dense(64, activation='relu'),
#       layers.Dense(1)
#     ])

#     model.compile(loss='mean_absolute_error',
#                 optimizer=tf.keras.optimizers.Adam(0.001))
#     return model

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=7))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 48        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse','mae'])

In [34]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2)

Epoch 1/100
1/1 - 1s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 2/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 3/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675837952.0000
Epoch 4/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 5/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 6/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 7/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675837952.0000
Epoch 8/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 9/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
E

Epoch 75/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 76/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675837952.0000
Epoch 77/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675837952.0000
Epoch 78/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675837952.0000
Epoch 79/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 80/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675838464.0000
Epoch 81/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 5675837952.0000
Epoch 82/100
1/1 - 0s - loss: 32310576348290613248.0000 - mse: 32310576348290613248.0000 - mae: 5675838464.0000
Epoch 83/100
1/1 - 0s - loss: 32310571950244102144.0000 - mse: 32310571950244102144.0000 - mae: 56758384